In [43]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [44]:
import xgboost as xgb
import lightgbm as gbm
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression

In [45]:
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, StratifiedKFold, GridSearchCV
from sklearn.metrics import f1_score, classification_report, confusion_matrix, make_scorer

from functools import partial

In [46]:
train = pd.read_parquet('../data/processed/train.parquet')
test = pd.read_parquet('../data/processed/test.parquet')

In [47]:
metric = partial(f1_score, average='macro')
score = make_scorer(f1_score, average='macro')

In [ ]:
X, y = train.loc[:, ~train.columns.isin(['revenue', "id"])], train.revenue
X_train, X_test, y_train, y_test = train_test_split(X,y, stratify=y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify=y_train, test_size=0.2, random_state=42)

In [91]:
import lightgbm as lgb
import numpy as np
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split

import optuna


# FYI: Objective functions can take additional arguments
# (https://optuna.readthedocs.io/en/stable/faq.html#objective-func-additional-args).
def objective(trial):
    data, target = sklearn.datasets.load_breast_cancer(return_X_y=True)
    train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.25)
    dtrain = lgb.Dataset(train_x, label=train_y)

    param = {
        "objective": "binary",
        "metric": "binary_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }

    gbm = lgb.train(param, dtrain)
    preds = gbm.predict(valid_x)
    pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.accuracy_score(valid_y, pred_labels)
    return accuracy

def objective_challenge(trial):

    param = {
        "n_estimators":trial.suggest_int("n_estimators", 300, 500, step=50),
        "objective": "binary",
        "random_state": 42,
        "verbosity": 0,
        "boosting_type": "goss",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "max_depth": trial.suggest_int("max_depth", 8,25, step=3),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
    }

    fit_params = {
        "eval_set":[(X_train, y_train), (X_val, y_val)], 
        "early_stopping_rounds":15,
        "eval_metric":[lambda y_true, y_pred: ("F1_MACRO", metric(y_true, np.round(y_pred)), True)],
        "feature_name":X_train.columns.tolist(),
        "categorical_feature":["Month", "TrafficType", "Browser", "SpecialDay", "OperatingSystems", "Region", "VisitorType", "Weekend"]
    }
    
    gbm = lgb.LGBMClassifier(**param, verbose=0)
    
    # In a better scenario, I might use this code.
#     f1_cross = cross_val_score(
#         gbm, X_train, y_train, 
#         cv=StratifiedKFold(3), 
#         scoring=score,
#         fit_params=fit_params
#     )
#     return np.mean(f1_cross)

    gbm.fit(X_train, y_train, **fit_params)
    
    return metric(y_val, gbm.predict(X_val))


In [92]:
study = optuna.create_study(direction="maximize")
study.optimize(objective_challenge, n_trials=50)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2021-02-06 21:38:28,164] A new study created in memory with name: no-name-51027e84-3555-4808-889d-ae00d6092cc3


[LightGBM] [Warning] lambda_l2 is set=3.95954785343799e-06, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.95954785343799e-06
[LightGBM] [Warning] verbosity is set=0, verbose=0 will be ignored. Current value: verbosity=0
[LightGBM] [Warning] lambda_l1 is set=8.01797280379652e-08, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.01797280379652e-08
[LightGBM] [Warning] lambda_l2 is set=3.95954785343799e-06, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.95954785343799e-06
[LightGBM] [Warning] verbosity is set=0, verbose=0 will be ignored. Current value: verbosity=0
[LightGBM] [Warning] lambda_l1 is set=8.01797280379652e-08, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.01797280379652e-08
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.061406 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] lambda_l2 

[I 2021-02-06 21:38:30,136] Trial 0 finished with value: 0.8018760362047372 and parameters: {'n_estimators': 500, 'lambda_l1': 8.01797280379652e-08, 'lambda_l2': 3.95954785343799e-06, 'max_depth': 23, 'num_leaves': 102}. Best is trial 0 with value: 0.8018760362047372.


[LightGBM] [Warning] lambda_l2 is set=0.3476724926731275, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.3476724926731275
[LightGBM] [Warning] verbosity is set=0, verbose=0 will be ignored. Current value: verbosity=0
[LightGBM] [Warning] lambda_l1 is set=4.778864026706593e-08, reg_alpha=0.0 will be ignored. Current value: lambda_l1=4.778864026706593e-08
[LightGBM] [Warning] lambda_l2 is set=0.3476724926731275, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.3476724926731275
[LightGBM] [Warning] verbosity is set=0, verbose=0 will be ignored. Current value: verbosity=0
[LightGBM] [Warning] lambda_l1 is set=4.778864026706593e-08, reg_alpha=0.0 will be ignored. Current value: lambda_l1=4.778864026706593e-08
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001109 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] lambda_l2 is s

[I 2021-02-06 21:38:32,326] Trial 1 finished with value: 0.7991209208196306 and parameters: {'n_estimators': 400, 'lambda_l1': 4.778864026706593e-08, 'lambda_l2': 0.3476724926731275, 'max_depth': 23, 'num_leaves': 106}. Best is trial 0 with value: 0.8018760362047372.


...

[LightGBM] [Warning] lambda_l2 is set=0.327380857357515, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.327380857357515
[LightGBM] [Warning] verbosity is set=0, verbose=0 will be ignored. Current value: verbosity=0
[LightGBM] [Warning] lambda_l1 is set=0.008613974430218257, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.008613974430218257
[LightGBM] [Warning] lambda_l2 is set=0.327380857357515, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.327380857357515
[LightGBM] [Warning] verbosity is set=0, verbose=0 will be ignored. Current value: verbosity=0
[LightGBM] [Warning] lambda_l1 is set=0.008613974430218257, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.008613974430218257
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000494 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] lambda_l2 is set=0.327

[I 2021-02-06 21:40:06,523] Trial 48 finished with value: 0.7929420862826796 and parameters: {'n_estimators': 500, 'lambda_l1': 0.008613974430218257, 'lambda_l2': 0.327380857357515, 'max_depth': 20, 'num_leaves': 160}. Best is trial 16 with value: 0.8148873610930947.


[LightGBM] [Warning] lambda_l2 is set=8.609100631927998, reg_lambda=0.0 will be ignored. Current value: lambda_l2=8.609100631927998
[LightGBM] [Warning] verbosity is set=0, verbose=0 will be ignored. Current value: verbosity=0
[LightGBM] [Warning] lambda_l1 is set=0.0016108530283158262, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0016108530283158262
[LightGBM] [Warning] lambda_l2 is set=8.609100631927998, reg_lambda=0.0 will be ignored. Current value: lambda_l2=8.609100631927998
[LightGBM] [Warning] verbosity is set=0, verbose=0 will be ignored. Current value: verbosity=0
[LightGBM] [Warning] lambda_l1 is set=0.0016108530283158262, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0016108530283158262
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000438 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] lambda_l2 is set=8

[I 2021-02-06 21:40:07,743] Trial 49 finished with value: 0.8212756627253428 and parameters: {'n_estimators': 400, 'lambda_l1': 0.0016108530283158262, 'lambda_l2': 8.609100631927998, 'max_depth': 17, 'num_leaves': 23}. Best is trial 49 with value: 0.8212756627253428.


Number of finished trials: 50
Best trial:
  Value: 0.8212756627253428
  Params: 
    n_estimators: 400
    lambda_l1: 0.0016108530283158262
    lambda_l2: 8.609100631927998
    max_depth: 17
    num_leaves: 23


In [94]:
trial.params

{'n_estimators': 400,
 'lambda_l1': 0.0016108530283158262,
 'lambda_l2': 8.609100631927998,
 'max_depth': 17,
 'num_leaves': 23}

In [87]:
# trial.distributions

In [29]:
# optuna.visualization.plot_optimization_history(study)

In [62]:
list(map(lambda t: t.values, study.get_trials()))

[[0.7909550045913682],
 [0.7902293265960547],
 [0.7860566159306097],
 [0.7788853293137588],
 [0.7896523544377526],
 [0.7843181655989415],
 [0.7850462304409673],
 [0.7834140330214399],
 [0.7882321429427727],
 [0.7881224236079303],
 [0.777307743502712],
 [0.7804572034975625],
 [0.7820968242875999],
 [0.8041321581239909],
 [0.7835839598997494],
 [0.7812057616568895],
 [0.7985389458891213],
 [0.8032792059858479],
 [0.7922613798284752],
 [0.7803906015329595],
 [0.7888305590426661],
 [0.7940695262498197],
 [0.7859621581426093],
 [0.7947752119651128],
 [0.7963169492752873],
 [0.7995478814542962],
 [0.7859621581426093],
 [0.7850462304409673],
 [0.7844056597592222],
 [0.7859621581426093]]

In [80]:
study.get_trials()[18]

FrozenTrial(number=18, values=[0.7922613798284752], datetime_start=datetime.datetime(2021, 2, 6, 20, 44, 24, 16981), datetime_complete=datetime.datetime(2021, 2, 6, 20, 44, 37, 328791), params={'n_estimators': 400, 'lambda_l1': 2.0621493119909519e-07, 'lambda_l2': 1.8787004833520626e-05, 'max_depth': 20, 'num_leaves': 15}, distributions={'n_estimators': IntUniformDistribution(high=500, low=300, step=50), 'lambda_l1': LogUniformDistribution(high=10.0, low=1e-08), 'lambda_l2': LogUniformDistribution(high=10.0, low=1e-08), 'max_depth': IntUniformDistribution(high=20, low=8, step=3), 'num_leaves': IntUniformDistribution(high=256, low=2, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=18, state=TrialState.COMPLETE, value=None)

In [95]:
fit_params = {
    "eval_set":[(X_train, y_train), (X_val, y_val)], 
    "early_stopping_rounds":15,
    "eval_metric":[lambda y_true, y_pred: ("F1_MACRO", metric(y_true, np.round(y_pred)), True)],
    "feature_name":X_train.columns.tolist(),
    "categorical_feature":["Month", "TrafficType", "Browser", "SpecialDay", "OperatingSystems", "Region", "VisitorType", "Weekend"]
}
    
gbm = lgb.LGBMClassifier(**trial.params, verbose=0)
    
gbm.fit(X, y, **fit_params)

/home/jloscalzo/.pyenv/versions/default_3.7.9/lib/python3.7/site-packages/lightgbm/basic.py:1551: UserWarning:

Using categorical_feature in Dataset.

/home/jloscalzo/.pyenv/versions/default_3.7.9/lib/python3.7/site-packages/lightgbm/basic.py:1555: UserWarning:

categorical_feature in Dataset is overridden.
New categorical_feature is ['Browser', 'Month', 'OperatingSystems', 'Region', 'SpecialDay', 'TrafficType', 'VisitorType', 'Weekend']



[LightGBM] [Warning] lambda_l1 is set=0.0016108530283158262, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0016108530283158262
[LightGBM] [Warning] lambda_l2 is set=8.609100631927998, reg_lambda=0.0 will be ignored. Current value: lambda_l2=8.609100631927998
[LightGBM] [Warning] lambda_l1 is set=0.0016108530283158262, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0016108530283158262
[LightGBM] [Warning] lambda_l2 is set=8.609100631927998, reg_lambda=0.0 will be ignored. Current value: lambda_l2=8.609100631927998
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.113442 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.


/home/jloscalzo/.pyenv/versions/default_3.7.9/lib/python3.7/site-packages/lightgbm/basic.py:1286: UserWarning:

Overriding the parameters from Reference Dataset.

/home/jloscalzo/.pyenv/versions/default_3.7.9/lib/python3.7/site-packages/lightgbm/basic.py:1098: UserWarning:

categorical_column in param dict is overridden.



[LightGBM] [Warning] lambda_l1 is set=0.0016108530283158262, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0016108530283158262
[LightGBM] [Warning] lambda_l2 is set=8.609100631927998, reg_lambda=0.0 will be ignored. Current value: lambda_l2=8.609100631927998
[LightGBM] [Warning] lambda_l1 is set=0.0016108530283158262, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0016108530283158262
[LightGBM] [Warning] lambda_l2 is set=8.609100631927998, reg_lambda=0.0 will be ignored. Current value: lambda_l2=8.609100631927998
[1]	valid_0's binary_logloss: 0.390802	valid_0's F1_MACRO: 0.458158	valid_1's binary_logloss: 0.387928	valid_1's F1_MACRO: 0.458219
Training until validation scores don't improve for 15 rounds
[2]	valid_0's binary_logloss: 0.362471	valid_0's F1_MACRO: 0.458158	valid_1's binary_logloss: 0.359485	valid_1's F1_MACRO: 0.458219
[3]	valid_0's binary_logloss: 0.340426	valid_0's F1_MACRO: 0.458158	valid_1's binary_logloss: 0.337514	valid_1's F1_MACRO: 0.458219
[

LGBMClassifier(lambda_l1=0.0016108530283158262, lambda_l2=8.609100631927998,
               max_depth=17, n_estimators=400, num_leaves=23, verbose=0)

In [96]:
metric(y, gbm.predict(X))

0.8800621867183407

In [99]:
ids = test.id
test.drop(columns=["id"], inplace=True)

In [101]:
result = gbm.predict(test)

In [102]:
from datetime import datetime
get_date = lambda : datetime.now().strftime("%Y%m%d_%H%M%S")

In [103]:
pd.DataFrame({"id":ids, "revenue":result}).to_csv(f'../data/results/{get_date()}_lgb_opt.csv', index=False)